In [1]:
import os
from openai import OpenAI
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
original_data = pd.read_csv("./Q2_20230202_majority_top_30.csv")[['tweet', 'label_true']]

def generate_tweet(twt, label):
    prompt = (
        f"Here is an example of a tweet about COVID-19 vaccination with a '{label}' stance:\n"
        f"Original Tweet: {twt}\n\n"
        f"Now make a creative modification to the tweet while keeping the stance '{label}'."
        f"Make sure the stance of the tweet is '{label}', and only answer in this format: \"Tweet: <modified_tweet> Stance: <stance>\"."
    )
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a creative person that likes to tweet about COVID-19 vaccination. You will come up with new tweets every time which vary in length. Always respond with the format: Tweet: <modified_tweet> Stance: <stance>."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=150,
        n=1,
        stop=None,
        temperature=0.65,  
        top_p=0.95
    )
    content = response.choices[0].message.content.strip()
    tweet = content.split("Tweet: ")[1].split(" Stance:")[0].strip()
    label = content.split("Stance: ")[1].strip()
    return tweet, label

tweets = []
labels = []
stances = {'in-favor': 10, 'against': 24, 'neutral-or-unclear': 37}

for _ , row in original_data.iterrows():
    og_tweet = row['tweet']
    stance = row['label_true']
    tweets.append(og_tweet)
    labels.append(stance)
    for _ in range(stances.get(stance, 1)):
        tweet, label = generate_tweet(og_tweet, stance)
        tweets.append(tweet)
        labels.append(label)

df = pd.DataFrame({
    'tweet': tweets,
    'label_true': labels
})
df.to_csv('aug_data1.csv', index=False)
